# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 50
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten
minute_interval = 15 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus

In [3]:
import distributionAlgorithmPolling
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecast
import distributionAlgorithmForecastPolling
import postOptimization
import numpy as np

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()

# Auswahl des Algorithmus
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_FIFO
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_EVEN_DISTRIBUTION
# distributionAlgorithmPolling.start_simulation(solarpeakleistung, maximale_ladezeit, ladeleistung_pro_bev, simulation_day, bev_data, table_dict, simulation_data, algorithm, minute_interval)

#distributionAlgorithmForecast.start_algorithm(simulation_data, simulation_day, maximale_ladezeit, solarpeakleistung, 
  #                  bev_data, table_dict, ladeleistung_pro_bev, minute_interval)

#distributionAlgorithmForecastPolling.start_simulation(solarpeakleistung, ladeleistung_pro_bev,
     #                simulation_day, bev_data, table_dict, simulation_data, minute_interval)
    
day_in_minute_interval_steps = list(np.around(np.arange(480, 960 + 1, minute_interval), 1))
distributionAlgorithmForecastPolling.init_simulation(day_in_minute_interval_steps, minute_interval, simulation_data, 
                                                     simulation_day, solarpeakleistung)

postOptimization.start_post_optimization(minute_interval, simulation_day, solarpeakleistung, bev_data, table_dict, simulation_data,
                    ladeleistung_pro_bev)
    
# nochmal ein Durchgang wo frei gewordener Platz belegt wird

START POST OPTIMIZATION
{0: [(8.5, 5.8), 'nicht parkend', [], [8.3925]], 1: [(8.5, 6.2), 'nicht parkend', [], [8.6175]], 2: [(8.5, 4.1), 'nicht parkend', [], [6.494999999999999]], 3: [(8.5, 6.1), 'nicht parkend', [], [8.6175]], 4: [(12.0, 4.0), 'nicht parkend', [], [4.09]], 5: [(8.5, 7.5), 'nicht parkend', [], [9.3725]], 6: [(9.0, 4.4), 'nicht parkend', [], [6.987499999999999]], 7: [(13.0, 3.0), 'nicht parkend', [], [2.5875]], 8: [(12.5, 3.5), 'nicht parkend', [], [3.195]], 9: [(8.0, 4.0), 'nicht parkend', [], [5.484999999999999]], 10: [(13.0, 3.0), 'nicht parkend', [], [2.5875]], 11: [(13.0, 3.0), 'nicht parkend', [], [2.5875]], 12: [(8.0, 4.9), 'nicht parkend', [], [6.987499999999999]], 13: [(12.0, 4.0), 'nicht parkend', [], [4.09]], 14: [(12.0, 3.0), 'nicht parkend', [], [3.53]], 15: [(12.5, 3.5), 'nicht parkend', [], [3.195]], 16: [(8.0, 5.1), 'nicht parkend', [], [7.229999999999999]], 17: [(12.5, 3.5), 'nicht parkend', [], [3.195]], 18: [(8.0, 5.9), 'nicht parkend', [], [8.0499999

Restladezeit:  16.922727272727244
AVAILABLE SOLAR POWER:  33.0
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 15:  2.7650000000000157
Restladezeit:  15.081818181818267
Charging BEVs:  [11, 13, 15]


Minute:  855
Waiting BEVs (after updating new waiting BEVs):  [19, 23, 24, 27, 29, 30, 31, 38, 39, 41, 42, 48, 14, 21]
Charging BEVs:  [13, 15, 17]
Waiting BEVs:  [19, 23, 24, 27, 29, 30, 31, 38, 39, 41, 42, 48, 14, 21]
Ladezeit = 21.58333333333337 für ID BEV 13
Ladezeit = 17.86666666666656 für ID BEV 15
Ladezeit = 13.159090909090878 für ID BEV 17
Ladezeit = 21.58333333333337 für ID BEV 13
Ladezeit = 17.86666666666656 für ID BEV 15
Ladezeit = 13.159090909090878 für ID BEV 17
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  4
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  4
NUMBER OF FREE CHARGING STATIONS:  1
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  4
AVAILABLE SOLAR POWER:  39.0
Restladeenergie bis zur fairen Lad

Restladezeit:  17.303129251701456
AVAILABLE SOLAR POWER:  36.75
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 42:  2.569375000000006
Restladezeit:  16.779591836734735
AVAILABLE SOLAR POWER:  36.75
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 48:  1.9568750000000117
Restladezeit:  12.77959183673477
Die Restladezeit des BEVS 48 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  14
Ladestart für ausgewähltes BEV:  782.2204081632652
9.1875 CHARGING POWER WIRD FÜR NEUES BEV 14 GESPEICHERT: 
ZEITPUNKT FÜR DAS ES GESPEICHERT WIRD:  782.2204081632652
Charging BEVs:  [30, 31, 42, 48]


Minute:  780
Waiting BEVs (after updating new waiting BEVs):  [21, 28, 37, 22, 46]
Charging BEVs:  [30, 31, 42, 14]
Waiting BEVs:  [21, 28, 37, 22, 46]
Ladezeit = 48.098585858586034 für ID BEV 30
Ladezeit = 46.09777777777754 für ID BEV 31
Ladezeit = 24.541176470588198 für ID BEV 42
Ladezeit = 2.2204081632652333 für ID BEV 14
Ladezeit = 48.09858585858

AVAILABLE SOLAR POWER:  53.12
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 21:  6.664757965006407
Restladezeit:  37.639822844539204
AVAILABLE SOLAR POWER:  53.12
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 37:  6.340300065004541
Restladezeit:  35.807417535793725
AVAILABLE SOLAR POWER:  53.12
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 46:  4.870342834131095
Restladezeit:  27.505701246975313
AVAILABLE SOLAR POWER:  53.12
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 20:  5.56528681451742
Restladezeit:  31.430460172349896
AVAILABLE SOLAR POWER:  53.12
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 9:  4.525999999999998
Restladezeit:  25.560993975903607
Charging BEVs:  [21, 37, 46, 20, 9]


Minute:  690
Waiting BEVs (after updating new waiting BEVs):  []
Charging BEVs:  [21, 37, 46, 20, 9]
Waiting BEVs:  []
Ladezeit = 77.01874397966185 für ID BEV 21
Ladezeit = 62.165724592866354 für ID BEV 37
Ladezeit = 33.13221635290347 für 

Number of charging stations:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
Charging BEVs:  []


Minute:  585
Waiting BEVs (after updating new waiting BEVs):  []
Charging BEVs:  []
Waiting BEVs:  []
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  4
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  4
NUMBER OF FREE CHARGING STATIONS:  4
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  4
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  4
Charging BEVs:  []


Minute:  570
Waiting BEVs (after updating new waiting BEVs):  []
Charging BEVs:  []
Waiting BEVs:  []
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  3
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  3
NUMBER OF FREE CHARGING STATIONS:  3
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  3
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  3
Charging BEVs:  []

### Ergebnis Algorithmus

In [4]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag)

>**Ergebnis**<br>Geladene BEVs: 29 von 50<br>Aufgeladene Solarenergie insgesamt: 127.62 kWh<br>Ungenutzte Solarenergie insgesamt: 68.7175 kWh

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [5]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict, minute_interval)

interactive(children=(IntSlider(value=480, description='Minute: ', max=960, min=480, step=15), Output()), _dom…

In [6]:
import figureGeneration

figureGeneration.create_charging_power_figure(simulation_day, solarpeakleistung, bev_data, minute_interval)

In [7]:
figureGeneration.create_bev_number_figure(simulation_data)

In [8]:
import postOptimization

postOptimization.start_post_optimization(minute_interval, simulation_day, solarpeakleistung, bev_data, table_dict, simulation_data,
                            ladeleistung_pro_bev)

START POST OPTIMIZATION
{0: [(8.5, 5.8), 'nicht parkend', [], [8.3925]], 1: [(8.5, 6.2), 'nicht parkend', [], [8.6175]], 2: [(8.5, 4.1), 'nicht parkend', [], [6.494999999999999]], 3: [(8.5, 6.1), 'nicht parkend', [], [8.6175]], 4: [(12.0, 4.0), 'nicht parkend', [(960, 30.10259917920655, 4.09)], [4.09]], 5: [(8.5, 7.5), 'nicht parkend', [(945, 68.41666666666663, 9.3725)], [9.3725]], 6: [(9.0, 4.4), 'nicht parkend', [], [6.987499999999999]], 7: [(13.0, 3.0), 'nicht parkend', [(929.8974008207935, 20.389106981930922, 2.5874999999999995)], [2.5875]], 8: [(12.5, 3.5), 'nicht parkend', [(909.5082938388625, 20.122579553148285, 3.1949999999999994)], [3.195]], 9: [(8.0, 4.0), 'nicht parkend', [(710.2905479955866, 60.95465256316061, 5.841323280617872)], [5.484999999999999]], 10: [(13.0, 3.0), 'nicht parkend', [(889.3857142857142, 16.519047619047683, 2.5875)], [2.5875]], 11: [(13.0, 3.0), 'nicht parkend', [(885, 16.840909090909122, 2.5875)], [2.5875]], 12: [(8.0, 4.9), 'nicht parkend', [], [6.9874

AVAILABLE SOLAR POWER:  27.0
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 5:  1.262500000000001
Restladezeit:  8.416666666666673
Die Restladezeit des BEVS 5 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  13
Ladestart für ausgewähltes BEV:  876.5833333333334
OVERWRITE INITIAL CHARGING DATA
9.0 CHARGING POWER WIRD FÜR NEUES BEV 13 GESPEICHERT: 
ZEITPUNKT FÜR DAS ES GESPEICHERT WIRD:  876.5833333333334
AVAILABLE SOLAR POWER:  27.0
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 10:  1.8200000000000074
Restladezeit:  12.133333333333383
Die Restladezeit des BEVS 10 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  15
Ladestart für ausgewähltes BEV:  872.8666666666666
OVERWRITE INITIAL CHARGING DATA
9.0 CHARGING POWER WIRD FÜR NEUES BEV 15 GESPEICHERT: 
ZEITPUNKT FÜR DAS ES GESPEICHERT WIRD:  872.8666666666666
AVAILABLE SOLAR POWER:  27.0
Restladeenergie bis zur fairen Ladeenergie for BE



Minute:  795
Waiting BEVs (after updating new waiting BEVs):  [14, 21, 28, 37]
Charging BEVs:  [30, 31, 42, 48]
Waiting BEVs:  [14, 21, 28, 37]
Ladezeit = 33.098585858586034 für ID BEV 30
Ladezeit = 31.097777777777537 für ID BEV 31
Ladezeit = 9.541176470588198 für ID BEV 42
Ladezeit = 3.9568627450979648 für ID BEV 48
Ladezeit = 33.098585858586034 für ID BEV 30
Ladezeit = 31.097777777777537 für ID BEV 31
Ladezeit = 9.541176470588198 für ID BEV 42
Ladezeit = 3.9568627450979648 für ID BEV 48
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  4
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  4
AVAILABLE SOLAR POWER:  36.75
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 30:  2.5503901515150638
Restladezeit:  16.65560915275144
AVAILABLE SOLAR POWER:  36.75
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 31:  2.649541666666786
Restladezeit:  17.303129251701456
AVAILABLE SOLAR POWER:  36.75
Restladeenergie bis zur fairen Ladeenergie f

AVAILABLE SOLAR POWER:  54.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 21:  7.030688957053111
Restladezeit:  38.786441469583174
AVAILABLE SOLAR POWER:  54.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 28:  2.1067868145174273
Restladezeit:  11.62258264720905
Die Restladezeit des BEVS 28 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  20
Ladestart für ausgewähltes BEV:  708.377417352791
OVERWRITE INITIAL CHARGING DATA
10.876000000000001 CHARGING POWER WIRD FÜR NEUES BEV 20 GESPEICHERT: 
ZEITPUNKT FÜR DAS ES GESPEICHERT WIRD:  708.377417352791
AVAILABLE SOLAR POWER:  54.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 37:  6.732873742871449
Restladezeit:  37.14347412396901
AVAILABLE SOLAR POWER:  54.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 22:  1.759999999999999
Restladezeit:  9.70945200441338
Die Restladezeit des BEVS 22 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewor

AVAILABLE SOLAR POWER:  50.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 20:  2.3012443186824836
Restladezeit:  10.962656539972134
Die Restladezeit des BEVS 20 endet vor dem nächsten Interval
Charging BEVs:  [21, 37, 46, 20]


Minute:  630
Waiting BEVs (after updating new waiting BEVs):  []
Charging BEVs:  [21, 37]
Waiting BEVs:  []
Ladezeit = 137.01874397966185 für ID BEV 21
Ladezeit = 122.16572459286635 für ID BEV 37
Ladezeit = 137.01874397966185 für ID BEV 21
Ladezeit = 122.16572459286635 für ID BEV 37
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
NUMBER OF FREE CHARGING STATIONS:  3
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
AVAILABLE SOLAR POWER:  50.62
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 21:  3.3334145817287935
Restladezeit:  7.902207621640759
Die Restlade